Roosevelt University

https://www.roosevelt.edu/contact/directory?txtSearch=&ddlGroupSearch%5B%5D=1&ddlCollegeSearch=All&ddlInstrumentSearch=All&ddlDepartmentSearch=678

**Libraries**

In [28]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup

**Initializing an instance of Google Chrome.**

In [29]:
# Initialize function
service = Service()

# Set preferences for Chrome
options = webdriver.ChromeOptions()

# Start
driver = webdriver.Chrome(service=service, options=options)


In [30]:
url = "https://www.roosevelt.edu/contact/directory?txtSearch=&ddlGroupSearch%5B%5D=1&ddlCollegeSearch=All&ddlInstrumentSearch=All&ddlDepartmentSearch=678"
driver.get(url)

driver.implicitly_wait(10)  # Wait 10 seconds 

In [31]:
# Get the HTML of the page after it has loaded completely
html = driver.page_source

# Parse the HTML using BeautifulSoup
soup = BeautifulSoup(html, "html.parser")


**Finding the professors' names with BS.**

In [32]:
# Find all divs with class "indFacultyListing-infoContainer"
divs_professors = soup.find_all("div", class_="indFacultyListing-infoContainer")

# List to store the data of professors
data_professors = []

In [44]:
data_professors = []


for div_professor in divs_professors:

    #NAME
    nome_professor = div_professor.find("div", class_="indDirectoryListing-name").find("span").get_text(strip=True)
    
    #LINK
    link_professor = div_professor.find("a", title="View {}'s Faculty Profile".format(nome_professor.strip())).get("href")
    
    # Get the full URL 
    full_link_professor = "https://www.roosevelt.edu" + link_professor
    
    # Open the professor's profile page with Selenium
    driver.get(full_link_professor)
    
    # Wait for the page to load completely
    driver.implicitly_wait(10)
    
    # Get the HTML of the profile page after it has loaded completely
    profile_html = driver.page_source
    
    # Parse the HTML using BeautifulSoup
    profile_soup = BeautifulSoup(profile_html, "html.parser")

    # COLLEGE
    college_name_span = profile_soup.find("span", id="collegeName")
    college_prefessor = college_name_span.find_next_sibling("div").get_text(strip=True) if college_name_span else "NaN"
    
    #EMAIL
    email_prefessor = profile_soup.find('div', class_='field_email').find('a').text.strip()
        
    data_professors.append({
        "Name": nome_professor,
        "Link": full_link_professor,
        "College": college_prefessor,
        "Email": email_prefessor,
    })


df_professors = pd.DataFrame(data_professors)

print(df_professors)
   


                     Name                                               Link  \
0          Evgeny Dantsin         https://www.roosevelt.edu/profile/edantsin   
1  Saba Jamaliannasrabadi  https://www.roosevelt.edu/profile/sjamaliannas...   
2            Michael Ruth            https://www.roosevelt.edu/profile/mruth   
3           Rami Salahieh        https://www.roosevelt.edu/profile/msalahieh   
4            Alex Wolpert         https://www.roosevelt.edu/profile/awolpert   
5              Ray Wright          https://www.roosevelt.edu/profile/rwright   

                                College                             Email  
0  » \n    \n          Computer Science            edantsin@roosevelt.edu  
1  » \n    \n          Computer Science  sjamaliannasrabadi@roosevelt.edu  
2  » \n    \n          Computer Science               mruth@roosevelt.edu  
3  » \n    \n          Computer Science           msalahieh@roosevelt.edu  
4  » \n    \n          Computer Science            awolpert

**Remove extra spaces from the elements.**

In [42]:

#Function to remove extra space
def remove_extra_spaces(text):
    return ' '.join(text.split())

#Aply to college 
df_professors["College"] = df_professors["College"].apply(remove_extra_spaces)

print(df_professors)


                     Name                                               Link  \
0          Evgeny Dantsin         https://www.roosevelt.edu/profile/edantsin   
1  Saba Jamaliannasrabadi  https://www.roosevelt.edu/profile/sjamaliannas...   
2            Michael Ruth            https://www.roosevelt.edu/profile/mruth   
3           Rami Salahieh        https://www.roosevelt.edu/profile/msalahieh   
4            Alex Wolpert         https://www.roosevelt.edu/profile/awolpert   
5              Ray Wright          https://www.roosevelt.edu/profile/rwright   

              College                             Email  
0  » Computer Science            edantsin@roosevelt.edu  
1  » Computer Science  sjamaliannasrabadi@roosevelt.edu  
2  » Computer Science               mruth@roosevelt.edu  
3  » Computer Science           msalahieh@roosevelt.edu  
4  » Computer Science            awolpert@roosevelt.edu  
5  » Computer Science             rwright@roosevelt.edu  


**Saving in a DF for easy viewing**

In [43]:
# Salvar o DataFrame em um arquivo CSV
df_professors.to_csv("df_professors.csv", index=False)


In [20]:
# Create a Pandas DataFrame with the data of professors
df_professors = pd.DataFrame(data_professors)

# Save the DataFrame directly to an Excel sheet
with pd.ExcelWriter('df_professors.xlsx', engine='xlsxwriter') as writer:
    df_professors.to_excel(writer, sheet_name='Sheet1', index=False)
